### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
## Laboratory 1 Solution
### Cooperating with: Luca Lodesani (s346978)
### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯

In [6]:
import numpy as np

In [12]:
# Problem 1 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))


In [14]:
# Problem 2 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [18]:
# Problem 3 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

## Base Hill Climbing 

In [22]:
# --- Hill climbing setup ---
solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)

# fitness
def value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity
def is_valid(sol):
    # max 1 bag for each item
    if np.any(sol.sum(axis=0) > 1):
        return False

    # check weight constraints for each knapsack
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

def tweak(sol):
    new_solution = sol.copy()
    # randomly select knapsack and item
    k = rng.integers(0, NUM_KNAPSACKS)
    i = rng.integers(0, NUM_ITEMS)
    #tweak: flip random item in random knapsack
    new_solution[k, i] = not new_solution[k, i]
    return new_solution

# --- Hill climbing loop ---
iterations = 10000
for _ in range(iterations):

    new_solution = tweak(solution)

    # generate new solution if invalid
    if not is_valid(new_solution):
        continue
    
    # update solution if better or equal
    if value(new_solution) >= value(solution):
        solution = new_solution

# --- Output results ---
total_value = value(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Zaino {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Zaino {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")


Total value: 1109544
Zaino 1 value: 13621
Zaino 2 value: 13615
Zaino 3 value: 10725
Zaino 4 value: 10985
Zaino 5 value: 11663
Zaino 6 value: 11231
Zaino 7 value: 13153
Zaino 8 value: 13744
Zaino 9 value: 8323
Zaino 10 value: 13087
Zaino 11 value: 12668
Zaino 12 value: 15752
Zaino 13 value: 10388
Zaino 14 value: 11120
Zaino 15 value: 11438
Zaino 16 value: 8443
Zaino 17 value: 11446
Zaino 18 value: 10895
Zaino 19 value: 11031
Zaino 20 value: 12007
Zaino 21 value: 9810
Zaino 22 value: 13111
Zaino 23 value: 11487
Zaino 24 value: 10096
Zaino 25 value: 10266
Zaino 26 value: 8102
Zaino 27 value: 11266
Zaino 28 value: 11946
Zaino 29 value: 8905
Zaino 30 value: 13455
Zaino 31 value: 11242
Zaino 32 value: 10347
Zaino 33 value: 11886
Zaino 34 value: 10143
Zaino 35 value: 11878
Zaino 36 value: 10574
Zaino 37 value: 8329
Zaino 38 value: 12464
Zaino 39 value: 12941
Zaino 40 value: 7621
Zaino 41 value: 11886
Zaino 42 value: 9933
Zaino 43 value: 10569
Zaino 44 value: 9409
Zaino 45 value: 11635
Zaino 4

# Tabu Search Hill Climbing

In [19]:
NUM_SAMPLES = 10  # number of neighbors to sample

In [21]:
# --- Hill climbing setup ---
solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)

# fitness
def value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity
def is_valid(sol):
    # max 1 bag for each item
    if np.any(sol.sum(axis=0) > 1):
        return False

    # check weight constraints for each knapsack
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

def tweak(sol):
    new_solution = sol.copy()
    # randomly select knapsack and item
    k = rng.integers(0, NUM_KNAPSACKS)
    i = rng.integers(0, NUM_ITEMS)
    #tweak: flip random item in random knapsack
    new_solution[k, i] = not new_solution[k, i] 
    return new_solution

# --- Hill climbing loop ---
iterations = 10000
tabu_list = set()
for _ in range(iterations):

    possible_solutions = [tweak(solution) for _ in range(NUM_SAMPLES)]
    candidates = [sol for sol in possible_solutions if is_valid(sol) and sol.flatten().tobytes() not in tabu_list]

    # if no valid candidates, continue
    if not len(candidates):
        continue

    new_solution = max(candidates, key=value)
    
    # update solution if better or equal
    if value(new_solution) >= value(solution):
        tabu_list.add(solution.flatten().tobytes())
        solution = new_solution

# --- Output results ---
total_value = value(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Zaino {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Zaino {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")
    


Total value: 1596958
Zaino 1 value: 12201
Zaino 2 value: 16519
Zaino 3 value: 16118
Zaino 4 value: 18232
Zaino 5 value: 14817
Zaino 6 value: 15374
Zaino 7 value: 13557
Zaino 8 value: 15962
Zaino 9 value: 14801
Zaino 10 value: 15871
Zaino 11 value: 15830
Zaino 12 value: 17252
Zaino 13 value: 11291
Zaino 14 value: 12567
Zaino 15 value: 14162
Zaino 16 value: 17540
Zaino 17 value: 15677
Zaino 18 value: 16851
Zaino 19 value: 16518
Zaino 20 value: 15332
Zaino 21 value: 16220
Zaino 22 value: 18176
Zaino 23 value: 15081
Zaino 24 value: 16948
Zaino 25 value: 13154
Zaino 26 value: 13304
Zaino 27 value: 17792
Zaino 28 value: 16368
Zaino 29 value: 18938
Zaino 30 value: 16852
Zaino 31 value: 15163
Zaino 32 value: 13213
Zaino 33 value: 16583
Zaino 34 value: 15029
Zaino 35 value: 18423
Zaino 36 value: 15722
Zaino 37 value: 14637
Zaino 38 value: 18707
Zaino 39 value: 16682
Zaino 40 value: 15824
Zaino 41 value: 18109
Zaino 42 value: 15544
Zaino 43 value: 16909
Zaino 44 value: 16370
Zaino 45 value: 1673